NOTES ON MY SOLUTIONS:

*   For this assignment, I put all of the text files into a google drive and read them myself and inputted them in the methods where necessary, so you (professor) don't have to pass in the text files yourself anywhere.
*   I have also included descriptions of how I solved each part of the problem for clarification sake


In [ ]:
# Get 3-b grammar rules.
!wget -q -O 3-1b_fixed.txt "https://drive.google.com/uc?export=download&id=1KVhFPZVjptKRi8U1YCsU9Fr8FhbC3v1r"
PartB = open("/content/3-1b_fixed.txt")
partB = PartB.read()

# Get 3-c sentences.
!wget -q -O 3-1c_fixed.txt "https://drive.google.com/uc?export=download&id=1yd-ZotoCF_HRJTHMkNJgUBTWMGbLDWPJ"
PartC = open("/content/3-1c_fixed.txt")
partC = PartC.read()
partC = partC.split("\n")


# Get 3-e sentences.
!wget -q -O 3-1e_fixed.txt "https://drive.google.com/uc?export=download&id=1Lpw8c5eVRYm-e1ldd0L__b0m_Hb4OMGs"
PartE = open("/content/3-1e_fixed.txt")
partE = PartE.read()
partE = partE.split("\n")


# Get 3-e sentences.
!wget -q -O final-rules.txt "https://drive.google.com/uc?export=download&id=1MxEAkcA_s-_9nT2zC6HAGeK5pcubx_HH"
PartErules = open("/content/final-rules.txt")
fixed_rules_e = PartErules.read()




**HW #3a Question:** (30 points) Implement a CKY parser in your favorite programming language. The
program should read in a  le containing CFG rules, read in input sentences and produce
parse trees for each input sentence according to the grammar. If there are multiple
parses for a sentence according to the grammar, then the parser should produce a count
of the possible parses and print out all the parse trees.

**My Solution:** For my solution, I created a few different methods. The general method to solve this problem is called trees and takes in a sentence and a set of rules. It then uses a method called parser_table to create two 2d arrays (one to keep track of tags and one to keep track of pointers to tags in the other array). Once the tables are filled, it calls a method called backtracking to recursively follow the pointers and find the parse of the phrase or sentence.

In [ ]:
#3a

# given an input sentence, creates two 2d arrays to keep track of possible tag combinations of all words
# returns an array of tags and an array of pointers to where the tags came from
def parser_table(sentence, cfg_rules):
  words = sentence.split(" ")
  terminal_rules = []
  for item in cfg_rules:
    items = item.split(" ")
    terminal_rules.append(items)
  sen_length = len(words)
  # creates two 2d arrays
  tag_array = [[list() for i in range(sen_length)] for j in range((sen_length))]
  # pointer array keeps track of where each tag came from
  pointer_array = [[list() for i in range(sen_length)] for j in range((sen_length))]

  # fills in the diagonal with the nonterimals corrosponding terminals
  for j in range(0, sen_length):
    for a in range(len(terminal_rules)):
      if ((terminal_rules[a])[1]) == words[j]:
        tag_array[j][j].append((terminal_rules[a])[0])

  # fills in the top triangular portion of the array going col by col adding tags that match with words
  for words in range(1, sen_length):
    for i in range(sen_length-words):
      j = i+words
      for k in range(i, j):
        for b in range(len(terminal_rules)):
          if (len(terminal_rules[b]))>2:
            if (len((tag_array[i][k])) > 0) and (len((tag_array[k+1][j])) > 0):
              for a in range(0, len((tag_array[i][k]))):
                if (terminal_rules[b])[1] == ((tag_array[i][k])[a]):
                  for c in range(0, len((tag_array[k+1][j]))):
                    if (terminal_rules[b])[2] == (tag_array[k+1][j])[c]:
                      tag_array[i][j].append((terminal_rules[b])[0])
                      location1 = a
                      location2 = c
                      pointer_array[i][j].append(((i, k), (location1), (k+1, j), (location2)))

  # returns the tag array and the pointer array of the inputted sentence
  # Each pointer array cell has a list of the coordinates that it points to,
  #as well as an index of which tag in the pointed cell was used
  return(tag_array, pointer_array)


# This method is a recursive function that takes a starting location, recovers the tag of the location,
# then goes to a new cell that the current location points to
def backtracking(row, col, position):
  # each time we get a new tag, we will add it to a string called parse
  global parse
  parse = parse + " " + "("
  # find the pointer coordinates in the cell
  pointer1row = ((((pointer_array[row][col])[position])[0])[0])
  pointer1col = ((((pointer_array[row][col])[position])[0])[1])
  pointer1tag = ((pointer_array[row][col][position])[1])
  # finds the tag in the cell
  tag1 = (tag_array[pointer1row][pointer1col])[pointer1tag]

  # if the row = col, we know we are at a nonterminal
  if pointer1row == pointer1col:
    # adds the tag to the parse
    parse = parse + " " + (str("(" + tag1 + " " + words[pointer1row] + ")"))
  else:
    # adds the tag to the parse
    parse = parse + " " + (tag1)

  # if we hit a cell with a pointer, then we recursively call this function
  if len(pointer_array[pointer1row][pointer1col]) > 0:
    backtracking(pointer1row, pointer1col, pointer1tag)

  # because each tag came from two previous tags, we have to call the function on the other pointer too
  pointer2row = ((((pointer_array[row][col])[position])[2])[0])
  pointer2col = (((pointer_array[row][col])[position])[2])[1]
  pointer2tag = ((pointer_array[row][col][position])[3])
  tag2 = (tag_array[pointer2row][pointer2col])[pointer2tag]

  if pointer2row == pointer2col:
    parse = parse + " " + (str("(" + tag2 + " " + words[pointer2row] + ")"))
  else:
    parse = parse + " " + (tag2)
  if len(pointer_array[pointer2row][pointer2col]) > 0:
    backtracking(pointer2row, pointer2col, pointer2tag)
  parse = parse + (")")


# This method uses helper methods to create a parse tree for each sentence in the sentences1 text file
def trees(sentences1, rules):
  global parse
  global words
  global tag_array
  global pointer_array
  sentences = sentences1
  all_rules = rules
  for i in range(len(sentences)):
    sentence = sentences[i]
    sentence = sentence.strip()
    words = sentence.split(" ")
    # creates the 2d arrays for the cky parses
    tag_array = parser_table(sentence, all_rules)[0]
    pointer_array = parser_table(sentence, all_rules)[1]

    # uses backtracking to find all the parses
    parse_count = 0
    print("\n")
    print(sentence)
    for k in range(len(tag_array[0][len(words)-1])):
      parse_count = parse_count+1
      first_tag = (tag_array[0][len(words)-1])[k]
      parse = ""
      parse = parse + (first_tag)
      backtracking(0, (len(words)-1), k)
      print("parse" + str(k+1)+ ":")
      print(parse)
    print("parse count: " + str(parse_count))


'''
file2 = '/content/3-1c.txt'
sentences = open(file2, 'r')
sentences = sentences.read()
sentences = sentences.split("\n")
'''



'\nfile2 = \'/content/3-1c.txt\'\nsentences = open(file2, \'r\')\nsentences = sentences.read()\nsentences = sentences.split("\n")\n'

**HW #3b Question:** (10 points) Write a program to convert the grammar in  le \3-1b.txt" to Chomsky
Normal Form. Your program does not have to be general CNF convertor for all CFGs.
The grammar rules have the format left hand side symbol followed by a list of right hand
side symbols. \S" is the start symbol for the grammar. Words that are in lowercase are
terminal symbols of the grammar.

**My Solution**: For my solution, I created a method that reads in a list of rules one by one. First, it checks for any rules that are unit rules of length two replaces that rule with new rules that account for the old rule. Then it loops through the rules again to check for any rules with a right side greater than 2 and replaces it with new rules by creating new variables

In [ ]:
#3b:
'''
file1 = '/content/Final-Rules.txt'
rule_list = open(file1, 'r')
rule_list = rule_list.read()
'''
# method to convert the grammer to cfg format
def cfg_converter(rule_list):
  all_rules = []
  rule_list_split = rule_list.split("\n")
  for i in range(len(rule_list_split)):
    all_rules.append(rule_list_split[i])

  # loop to go through the list of rules and find ones of length two
  i = 0;
  while (True):
    if (i > (len(all_rules)-1)):
      break
    words = all_rules[i].split(" ")
    if len(words) == 2 and not words[1].islower():
      for j in range(len(all_rules)):
        cur_rule = all_rules[j].split(" ")
        # if we find a rule that begins with the nonterminal
        if words[1] == cur_rule[0]:
          # create a new rule to replace it
          if (len(cur_rule)) == 2:
            new_rule = words[0]+ " " + cur_rule[1]
          else:
            new_rule = words[0] + " " + cur_rule[1] + " " + cur_rule[2]
          all_rules.append(new_rule)
      all_rules.remove(all_rules[i])
    else:
      i = i+1

  # loop to go through the list of rules and find ones of length four
  i = 0
  v = 0
  while (True):
    var1 = "X"
    if (i > (len(all_rules)-1)):
      break
    words = all_rules[i].split(" ")
    if len(words) == 4:
      # creates a new variable and splits up the rule so it is of length 3
      new_rule = words[0] + " " + str(v) + " " + words[3]
      new_rule2 = str(v) + " " + words[1] + " " + words[2]
      all_rules.append(new_rule)
      all_rules.append(new_rule2)
      all_rules.remove(all_rules[i])
      v = v+1
    else:
      i = i+1
      v = v+1

  # deletes any repetative rules
  index = 0
  while (True):
    if (index > (len(all_rules)-1)):
      break
    if (all_rules.count(all_rules[index]) > 1):
      all_rules.remove(all_rules[index])
    index = index+1

  return all_rules


print(cfg_converter(partB))




['S NP VP', 'NP Det NBar', 'NP NP PP', 'NBar N NBar', 'PP Prep NP', 'VP V NP', 'VP VP PP', 'Prep with', 'Prep on', 'Prep in', 'Prep at', 'Prep like', 'Adj green', 'Adj light', 'N light', 'N car', 'N cars', 'N truck', 'N man', 'N baby', 'Det the', 'Det a', 'V fly', 'V hit', 'V gave', 'V put', 'V like', 'COMP that', 'CONJ and', 'NP N NBar', 'NBar light', 'NBar car', 'NBar cars', 'NBar truck', 'NBar man', 'NBar baby', 'VP fly', 'VP hit', 'VP gave', 'VP put', 'VP like', 'NP light', 'NP car', 'NP cars', 'NP truck', 'NP man', 'NP baby', 'NP 1 NBar', '1 Det Adj', 'VP 7 NP', '7 V NP', 'VP 8 PP', '8 V NP']


**Question: HW#C (20 points)** Using the converted grammar from 1(b) parse the sentences in \3-1c.txt"
using your CKY parser. Your program should output (a) the number of parses and (b)
all parse trees for each sentence. You may output parse trees as indented lines; you don't
have to draw pretty trees.

**My Solution:** To implement my solution, I simply converted the grammar from part B to CNF and then plugged in the sentences to my method from A to print out all the possible trees and parses

In [ ]:
#Part C

# calls the grammer converter and the method that creates the parsing trees
all_rules = cfg_converter(partB)
trees(partC, all_rules)



light cars fly
parse1:
S ( NP ( (N light) (NBar cars)) (VP fly))
parse count: 1


light green cars fly
parse count: 0


the man gave a baby a car
parse1:
S ( NP ( (Det the) (NBar man)) VP ( 7 ( (V gave) NP ( (Det a) (NBar baby))) NP ( (Det a) (NBar car))))
parse count: 1


the baby put the man on the truck
parse1:
S ( NP ( (Det the) (NBar baby)) VP ( (V put) NP ( NP ( (Det the) (NBar man)) PP ( (Prep on) NP ( (Det the) (NBar truck))))))
parse2:
S ( NP ( (Det the) (NBar baby)) VP ( VP ( (V put) NP ( (Det the) (NBar man))) PP ( (Prep on) NP ( (Det the) (NBar truck)))))
parse3:
S ( NP ( (Det the) (NBar baby)) VP ( 8 ( (V put) NP ( (Det the) (NBar man))) PP ( (Prep on) NP ( (Det the) (NBar truck)))))
parse count: 3


the baby hit the truck with the car
parse1:
S ( NP ( (Det the) (NBar baby)) VP ( (V hit) NP ( NP ( (Det the) (NBar truck)) PP ( (Prep with) NP ( (Det the) (NBar car))))))
parse2:
S ( NP ( (Det the) (NBar baby)) VP ( VP ( (V hit) NP ( (Det the) (NBar truck))) PP ( (Prep with)

**HW3d Question:** (20 points) Some sentences in \3-1c.txt" cannot be parsed by the grammar. Extend
your parser to recover the set of partial parses that span the sentence. Partial parses
are parses for substrings of the input. A partial parse that spans the most number of
tokens is a longest spanning partial parse. A set of partial parses is needed to span all
the tokens of a sentence. You can use a greedy approach to selecting the partial parse
to be included in the set.

**My Solution**: To implement my solution, I created a method to find a set of parses of a sentence with the longest partial parse. I first called the parse_table method to fill in all possible parses into a table and then I traversed the table from the top right cell to find the longest partial parse by looking at the cells that account for the most amount of words. Then, I called the backpointer method recursively to get a parse for the partial sentence. Then I recursively called my solution on the rest of the sentence (both sides that aren't acocunted for by the parse) The first parse in the list is the longest partial parse. If a sentence parses completely, I simply printed out the parse

In [ ]:
#3d:

# final_parse will eventually be a string with each longest possible partial parse
final_parse = ""

# This method is similar to trees, but now returns all partial parses. It calls 2 helper methods
def trees2(sentences1, rules):
  global parse
  global words
  global tag_array
  global pointer_array
  global final_parse
  sentence = sentences1
  all_rules = rules
  words = sentence.split(" ")
  # creates the 2d arrays for the given part of the sentence
  tag_array = parser_table(sentence, all_rules)[0]
  pointer_array = parser_table(sentence, all_rules)[1]

  # diagonally searches from the top right corner until it hits a cell with a tag
  k = 0
  var1 = True
  for i in reversed(range(len(words))):
    if (var1 == False):
      break
    k = k+1
    for l in range(0,k):
      if (var1 == False):
        break
      if (len(tag_array[l][i+l]) >0):

        # once it hits a cell with a tag, it finds the parse of the words accounted for by the cell
        var1 = False
        first_tag = (tag_array[l][i+l])[0]
        parse = ""
        parse = parse + (first_tag)
        # determines whether this tag is a terminal or a nonterminal
        if (l==i+l):
          final_parse = final_parse + " \nPartial parse: " + " (" + first_tag + " " + words[l] + ")"
        else:
          # if it is a nonterminal, it backtracks to find the parse
          final_parse = final_parse + " \nPartial parse: "
          backtracking(l, (i+l), 0)
          final_parse = final_parse + parse
        new_part = ""
        new_part2 = ""

        # To get the rest of the sentence, recurseively call this method on
        # every part of the sentence not accounted for by these words
        for v in range(0, l):
          if (v == 0):
            new_part = words[v]
          else:
            new_part = new_part + " " + words[v]
        for x in range(i+l+1, len(words)):
          if (x == 0):
            new_part2 = words[x]
          else:
            new_part2 = new_part2 + " " + words[x]
        if (len(new_part) >0):
          trees2(new_part, all_rules)
        if (len(new_part2) >0):
          trees2(new_part2, all_rules)
        return (final_parse)


# converts the rules
the_rules = cfg_converter(partB)

# calls the partial parser tree method on each sentence
for i in range(len(partC)):
  sentencetest = partC[i]
  print("sentence: " + sentencetest)
  print(trees2(sentencetest, the_rules))
  print("\n")
  print("\n")
  final_parse = ""


sentence: light cars fly
 
Partial parse: S ( NP ( (N light) (NBar cars)) (VP fly))




sentence: light green cars fly
 
Partial parse: S ( (NP cars) (VP fly)) 
Partial parse:  (Adj light) 
Partial parse:  (Adj green)




sentence: the man gave a baby a car
 
Partial parse: S ( NP ( (Det the) (NBar man)) VP ( 7 ( (V gave) NP ( (Det a) (NBar baby))) NP ( (Det a) (NBar car))))




sentence: the baby put the man on the truck
 
Partial parse: S ( NP ( (Det the) (NBar baby)) VP ( (V put) NP ( NP ( (Det the) (NBar man)) PP ( (Prep on) NP ( (Det the) (NBar truck))))))




sentence: the baby hit the truck with the car
 
Partial parse: S ( NP ( (Det the) (NBar baby)) VP ( (V hit) NP ( NP ( (Det the) (NBar truck)) PP ( (Prep with) NP ( (Det the) (NBar car))))))




sentence: the man at the light green light put the baby in a car
 
Partial parse: S ( (NP light) VP ( (V put) NP ( NP ( (Det the) (NBar baby)) PP ( (Prep in) NP ( (Det a) (NBar car)))))) 
Partial parse: NP ( NP ( (Det the) (NBar man))

**Question HW3(e) (20 points):** How will you \ x" the grammar in a general way so that the ungrammatical
sentences in \3-1e.txt" do not parse? Show the rules of the \ xed" grammar, run your
parser with the  xed grammar to show that the ungrammatical sentences do not parse
(produce their partial parses instead). Explain what you did to  x the grammar and the
rationale for what you did.

**My Solution:** To fix the grammar, I realized that these sentences have three problems. The first one needs to account for singular and plural differences. The second one needs to account for the fact that some verbs are transitive and only take one argument. The third sentence needs the rules to account for the fact that some verbs are ditransitive and take two arguements.

To address these needs, I assigned all N, NBar, and NP as singular or plural. I then assigned all revelent verbs as singular or plural as well. Then, I assigned the verbs as transitive or ditransitive. This made me have a lot more terminals in my rule list. Because of this, I now had to change all the subsequent rules so that Ns, NBars, and NPs were only matching up with Vs and VPs that had the same plurality as them. I also changed all subsequent rules so that V, and VP would only take the relevent amount of arguments. To do this, I needed to specify whether an NP had two arguments or one, which affected the VPs that it was allowed to combine with. In my results, it is clear that the partial parses couldn't combine with each other because of the reasons listed above.

In [ ]:
# Part e:

# converts the new rules to CNF
new_rules = cfg_converter(fixed_rules_e)

# goes through every sentence in parte and prints their partial parses
for i in range(len(partE)):
  sentencetest = partE[i]
  print("sentence: " + sentencetest)
  print(trees2(sentencetest, new_rules))
  print("\n")
  print("\n")
  final_parse = ""


sentence: the car fly
 
Partial parse: NP.sg.arg1 ( (Det the) (NBar.sg car)) 
Partial parse:  (V.pl fly)




sentence: the truck hit the baby the car
 
Partial parse: S ( NP.sg.arg1 ( (Det the) (NBar.sg truck)) VP.sg.trans ( (V.trans hit) NP.sg.arg1 ( (Det the) (NBar.sg baby)))) 
Partial parse: NP.sg.arg1 ( (Det the) (NBar.sg car))




sentence: the man put the baby
 
Partial parse: NP.sg.arg1 ( (Det the) (NBar.sg man)) 
Partial parse: NP.sg.arg1 ( (Det the) (NBar.sg baby)) 
Partial parse:  (V.ditrans put)




